### Free space

No use of symmetry factors, just use all elements
        
       symME.py

In [1]:
from symengine import *
import sympy
import numpy as np
import re
from joblib import Parallel, delayed

### Definitions and auxiliary functions

In [2]:
xij, yij, zij = symbols("xij, yij, zij", real=True) 
b, eta = symbols("b, eta", real=True, positive=True)

# indices for summation:
## Caution, cannot use lambda, instead use Lambda
## do not use eta as index, it's the viscosity
alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma = symbols(
    'alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma', integer=True)


## for FH and VH checkout the 'Indexed' package to define them as FH[1], FH[2] etc.

rij = sympy.Array([xij, yij, zij])

def norm(r):
    result=0
    for i in range(3):
        result += r[i]**2
    return sqrt(result)

KroneckerDelta = eye(3)

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

In [3]:
def Laplacian(G):
    return diff(G, rij[0], rij[0]) + diff(G, rij[1], rij[1]) + diff(G, rij[2], rij[2])

def Curl(alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(alpha,i,j)*diff(G(j,beta),rij[i]) ## G is symmetric in its indices
    return result

def tilde_Curl(alpha, beta): ##introduce for convenience
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(beta,i,j)*diff(G(alpha, j),rij[i])
    return result

def CurlCurl(alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(alpha,i,j)*eps(beta,k,l)*diff(G(j,l), rij[i], rij[k])
    return result


def SumG(sigma, eta2, mu): ##for symengine to work
    result=0
    for i in range(3):
        result += eps(sigma, eta2, i)*Laplacian( G(mu, i) )
    return result


def SumG3s1Sym(mu, nu2, alpha, gamma1, gamma2):
    result=0
    for i in range(3):
        result += eps(mu,nu2,i)*g3s1Sym(alpha,gamma1,gamma2,i)
    return result

### Matrix elements

In [4]:
def G(alpha, beta):
    return 1/(8*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij) + rij[alpha]*rij[beta]/norm(rij)**3)

def delG(alpha, beta, gamma):
    return 1/(8*pi*eta)*((-KroneckerDelta[alpha,beta]*rij[gamma] + KroneckerDelta[alpha,gamma]*rij[beta] + KroneckerDelta[beta,gamma]*rij[alpha])/norm(rij)**3
                         - 3*rij[alpha]*rij[beta]*rij[gamma]/norm(rij)**5)

def lapG(alpha, beta):
    return 1/(4*pi*eta)*(KroneckerDelta[alpha,beta]/norm(rij)**3 - 3*rij[alpha]*rij[beta]/norm(rij)**5)

In [5]:
##auxiliary functions
def dellapG(alpha, beta, kappa1):
    return diff(lapG(alpha, beta), rij[kappa1])

def deldelG(zeta, beta, kappa1, nu):
    return diff(delG(zeta, beta, kappa1), rij[nu])

def deldellapG(alpha, beta, gamma1, kappa1):
    return diff(dellapG(alpha,beta, gamma1), rij[kappa1])

def deldeldelG(alpha, beta, gamma1, gamma2, gamma3):
    return diff(deldelG(alpha, beta, gamma1, gamma2), rij[gamma3])

def deldeldeldelG(alpha, beta, gamma1, gamma2, gamma3, gamma4):
    return diff(deldeldelG(alpha, beta, gamma1, gamma2, gamma3), rij[gamma4])

In [6]:
# GLL
def G1s1s(alpha, beta):
    return sympy.simplify(sympy.nsimplify(G(alpha, beta) + b**2/3*lapG(alpha, beta)))

def G1s2a(alpha, beta):
    return sympy.simplify(sympy.nsimplify(-1/2*b*tilde_Curl(alpha,beta)))

def G2a1s(alpha, beta):
    return sympy.simplify(sympy.nsimplify(b*Curl(alpha,beta)))

def G2a2a(alpha, beta):
    return sympy.simplify(sympy.nsimplify(-1/2*b*b*CurlCurl(alpha,beta)))

In [7]:
## GL2s and G2sL
def G1s2s(alpha, kappa1, beta):
    return sympy.simplify(sympy.nsimplify(-1/2*b*((delG(alpha, beta, kappa1) + delG(alpha, kappa1, beta)) + 4*b**2/15*(dellapG(alpha, beta, kappa1) + dellapG(alpha, kappa1, beta)))))

def G2a2s(alpha, kappa1, beta):
    return sympy.simplify(sympy.nsimplify(-1/2*b*b*(diff(Curl(alpha, beta),rij[kappa1]) + diff(Curl(alpha, kappa1),rij[beta]))))

def G2s1s(alpha, gamma1, beta):
    return sympy.simplify(sympy.nsimplify(1/2*b*((delG(alpha, beta, gamma1) + delG(gamma1, beta, alpha)) + 4*b**2/15*(dellapG(alpha,beta,gamma1) + dellapG(gamma1, beta, alpha)))))

def G2s2a(alpha, gamma1, mu):
    return sympy.simplify(sympy.nsimplify(-1/4*b*b*(diff(tilde_Curl(gamma1,mu), rij[alpha]) + diff(tilde_Curl(alpha,mu), rij[gamma1])))) #sign change from -> eps(mu, kappa1, beta)


## G2s2s
def G2s2s(alpha, gamma1, kappa1, beta):
    return sympy.simplify(sympy.nsimplify(-1/4*b*b*((deldelG(alpha, beta, gamma1, kappa1) + deldelG(gamma1, beta, alpha, kappa1)) + 
                      (deldelG(alpha,kappa1, gamma1, beta) + deldelG(gamma1, kappa1, alpha, beta)) +
                      b**2/5*(deldellapG(alpha,beta,gamma1,kappa1) + deldellapG(gamma1,beta,alpha,kappa1)) + 
                      b**2/5*(deldellapG(alpha, kappa1, gamma1, beta) + deldellapG(gamma1,kappa1,alpha,beta)))))


## higher matrix elements
def G1s3t(alpha, beta):
    return sympy.simplify(sympy.nsimplify(-1/5*b**2*lapG(alpha,beta)))


# symmetry for matrix elements of form l sigma, 3s
def g13sSym(alpha,kappa2,kappa1,beta):
    return 1/3*(deldelG(alpha,beta,kappa1,kappa2) + deldelG(alpha,kappa2,kappa1,beta) + deldelG(alpha,kappa1,beta,kappa2)) - 1/15*(
        KroneckerDelta[kappa1,kappa2]*lapG(alpha,beta) + KroneckerDelta[kappa1,beta]*lapG(alpha,kappa2) + KroneckerDelta[beta,kappa2]*lapG(alpha,kappa1))

def G1s3s(alpha,kappa2, kappa1, beta):
    return sympy.simplify(sympy.nsimplify(b**2*(g13sSym(alpha,kappa2,kappa1,beta) + 5*b**2/21*Laplacian(g13sSym(alpha,kappa2,kappa1,beta)))))


# symmetry of matrix elements of form 3s, l sigma
def g3s1Sym(alpha,gamma1,gamma2,beta):
    return 1/3*(deldelG(alpha,beta,gamma1,gamma2) + deldelG(gamma1,beta,gamma2,alpha) + deldelG(gamma2,beta,alpha,gamma1)) - 1/15*(
        KroneckerDelta[gamma1, gamma2]*lapG(alpha,beta) + KroneckerDelta[gamma2,alpha]*lapG(gamma1,beta) + KroneckerDelta[alpha,gamma1]*lapG(gamma2,beta))

def G3s1s(alpha,gamma1,gamma2,beta):
    return sympy.simplify(sympy.nsimplify(b**2*(g3s1Sym(alpha,gamma1,gamma2,beta) + 5*b**2/21*Laplacian(g3s1Sym(alpha,gamma1,gamma2,beta)))))

def G3a1s(Lambda, kappa2, beta):
    return sympy.simplify(sympy.nsimplify(1/2*b**2*(diff(Curl(Lambda, beta), rij[kappa2]) + diff(Curl(kappa2,beta), rij[Lambda]))))

def G1s3a(alpha, nu2, sigma):
    return sympy.simplify(sympy.nsimplify(2/3*b**2*(diff(tilde_Curl(alpha,nu2),rij[sigma]) - 1/3*SumG(nu2,sigma,alpha)))) # sign change due to --> eps(sigma,nu1,beta)

def g3sCurlSym(alpha,gamma1,gamma2,mu):
    return 1/3*(diff(tilde_Curl(alpha,mu), rij[gamma1],rij[gamma2]) + diff(tilde_Curl(gamma1,mu), rij[gamma2],rij[alpha]) + diff(tilde_Curl(gamma2,mu), rij[alpha],rij[gamma1])) 
        

def G3s2a(alpha,gamma1,gamma2,mu):
    return sympy.simplify(sympy.nsimplify(-1/2*b**3*g3sCurlSym(alpha, gamma1, gamma2, mu)))## sign change due to -> eps(mu, kappa1, beta)

def G3a2a(Lambda, kappa2, mu):
    return sympy.simplify(sympy.nsimplify(-1/4*b**3*(diff(CurlCurl(Lambda,mu), rij[kappa2]) + diff(CurlCurl(kappa2,mu), rij[Lambda])))) #sign change -> eps(mu,kappa1,beta)

def G2s3t(alpha,gamma1,nu2):
    return sympy.simplify(sympy.nsimplify(-b**3/10*(dellapG(alpha,nu2,gamma1) + dellapG(gamma1,nu2,alpha))))

def G3s3t(alpha,gamma1,gamma2,nu2):
    return sympy.simplify(sympy.nsimplify(-b**4/5*Laplacian(g3s1Sym(alpha,gamma1,gamma2,nu2))))


def G2s3a(alpha,gamma1,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return sympy.simplify(sympy.nsimplify(b**3/3*( diff(tilde_Curl(alpha,mu), rij[gamma1], rij[nu2]) - 1/3*diff(SumG(mu,nu2,alpha), rij[gamma1])
                   + diff(tilde_Curl(gamma1,mu), rij[alpha], rij[nu2]) - 1/3*diff(SumG(mu,nu2,gamma1), rij[alpha]))))

def G3a3a(Lambda, kappa2, mu, nu2): #sign change -> eps(mu,nu1,beta)
    return sympy.simplify(sympy.nsimplify(b**4/3*(diff(CurlCurl(Lambda,mu), rij[nu2], rij[kappa2]) + diff(CurlCurl(kappa2,mu), rij[nu2], rij[Lambda]))))


def G3s3a(alpha,gamma1,gamma2,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return sympy.simplify(sympy.nsimplify(2*b**4/3*(diff(g3sCurlSym(alpha,gamma1,gamma2,mu), rij[nu2]) - 1/3*Laplacian(SumG3s1Sym(mu,nu2,alpha,gamma1,gamma2)))))


def g2s3sSym(alpha,gamma1,kappa2, kappa1, beta):
    return diff(g13sSym(alpha,kappa2,kappa1,beta), rij[gamma1]) + diff(g13sSym(gamma1, kappa2, kappa1, beta), rij[alpha])

def G2s3s(alpha,gamma1,kappa2, kappa1, beta):
    return sympy.simplify(sympy.nsimplify(1/2*b**3*(g2s3sSym(alpha,gamma1,kappa2, kappa1, beta) + 6*b**2/35*Laplacian(g2s3sSym(alpha,gamma1,kappa2, kappa1, beta)))))


def gCurl3sSym(tau, kappa2, kappa1, beta):
    return 1/3*(diff(Curl(tau, beta), rij[kappa1],rij[kappa2]) + diff(Curl(tau,kappa1), rij[kappa2],rij[beta]) + diff(Curl(tau,kappa2), rij[beta],rij[kappa1])) 


def G3a3s(Lambda, eta2, kappa2, kappa1, beta):
    return sympy.simplify(sympy.nsimplify(1/2*b**4*(diff(gCurl3sSym(Lambda, kappa1, kappa2, beta), rij[eta2]) + diff(gCurl3sSym(eta2, kappa1, kappa2, beta), rij[Lambda]))))


def deldelg13sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return diff(g13sSym(alpha,kappa2,kappa1,beta), rij[gamma1], rij[gamma2])

def lapg13sSym(alpha,kappa2,kappa1,beta):
    return Laplacian(g13sSym(alpha,kappa2,kappa1,beta))

def g3s3sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return 1/3*(deldelg13sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta) + deldelg13sSym(gamma1,gamma2,alpha,kappa2,kappa1,beta) + 
                deldelg13sSym(gamma2,alpha,gamma1,kappa2,kappa1,beta)) - 1/15*(
                KroneckerDelta[gamma1,gamma2]*lapg13sSym(alpha, kappa2, kappa1, beta) + 
                KroneckerDelta[gamma2,alpha]*lapg13sSym(gamma1, kappa2, kappa1, beta) + 
                KroneckerDelta[alpha,gamma1]*lapg13sSym(gamma2, kappa2, kappa1, beta))

def G3s3s(alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return sympy.simplify(sympy.nsimplify(b**4*(g3s3sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta) + b**2/7*Laplacian(g3s3sSym(alpha,gamma1,gamma2,kappa2,kappa1,beta)))))


def G3a2s(Lambda, kappa2, kappa1, beta):
    return sympy.simplify(sympy.nsimplify(-1/4*b**3*(diff(Curl(Lambda,beta), rij[kappa1], rij[kappa2]) + diff(Curl(Lambda,kappa1), rij[beta], rij[kappa2])
                       + diff(Curl(kappa2,beta), rij[kappa1], rij[Lambda]) + diff(Curl(kappa2,kappa1), rij[beta], rij[Lambda]))))


def g3s2sSym(alpha,gamma1,gamma2, kappa1, beta):
    return diff(g3s1Sym(alpha,gamma1,gamma2,beta), rij[kappa1]) + diff(g3s1Sym(alpha,gamma1,gamma2,kappa1), rij[beta])

def G3s2s(alpha,gamma1,gamma2,kappa1,beta):
    return sympy.simplify(sympy.nsimplify(-1/2*b**3*(g3s2sSym(alpha,gamma1,gamma2, kappa1, beta) + 6*b**2/35*Laplacian(g3s2sSym(alpha,gamma1,gamma2, kappa1, beta)))))


def G2a3a(Lambda, mu, nu2): # sign change -> eps(mu,nu1,beta)
    return sympy.simplify(sympy.nsimplify(2*b**3/3*diff(CurlCurl(Lambda, mu), rij[nu2])))


def G2a3s(Lambda,kappa2,kappa1,beta):
    return sympy.simplify(sympy.nsimplify(b**3*gCurl3sSym(Lambda, kappa1, kappa2, beta)))

In [8]:
## zero elements

def G3t1s(alpha,beta):
    return 0

def G3t2a(alpha,beta):
    return 0

def G3t2s(alpha, beta, kappa1):
    return 0

def G3t3t(alpha,beta):
    return 0

def G3t3a(alpha, beta, kappa1):
    return 0

def G3t3s(alpha, beta, kappa1, kappa2):
    return 0

def G2a3t(alpha,beta):
    return 0

def G3a3t(alpha, gamma1, beta):
    return 0

### Systematic method of identifying which symmetry factor has to be applied

In [9]:
def check_duplicates(listOfElems): ## for eg 0,0,1 but will also give true for 0,0,0
    ''' Check if given list contains any duplicates '''
    if len(listOfElems) == len(set(listOfElems)):
        return False
    else:
        return True

def all_equal(iterator): ## for 0,0,0
    return len(set(iterator)) <= 1 

In [10]:
## 3s template
## Number of x,y and z: 18x, 6y and 3z. Want to stop if as quick as possible --> check x, then y, then z
m,n,l = 1,1,2

if check_duplicates([m,n,l]) and not all_equal([m,n,l]): ## two indices are equal
    print('x')
elif not check_duplicates([m,n,l]): ## all indices different
    print('y')
else:                           ## diagonal, all indices equal
    print('z') 

x


In [11]:
## 2s template (same for 3a)
## Number of a and b: 6a and 3b
i,j = 0,1

if not all_equal([i,j]): ## off-diagonal terms
    print('a')
else:                    ## diagonal terms
    print('b')


a


In [26]:
##symmetry factors for some matrix elements
def symG1s2s(alpha, beta, kappa1):
    if beta == kappa1:
        return 0#sympy.simplify(3/2*G1s2s(alpha, beta, kappa1) - 3*G1s2s(alpha, 1-beta, 1-kappa1)) ##maybe remove simplify from those functions and introduce it here
    else:
        return sympy.simplify(2*G1s2s(alpha, beta, kappa1))
    
def symG1s3a(alpha, beta, kappa1):
    if beta == kappa1:
        return 0#sympy.simplify(2*G1s3a(alpha, beta, kappa1) + G1s3a(alpha, 1-beta, 1-kappa1)) 
    else:
        return sympy.simplify(2*G1s3a(alpha, beta, kappa1))
    
def symG1s3s(alpha, beta, kappa1, kappa2):
    if all_equal([beta,kappa1,kappa2]):
        return sympy.simplify(4*G1s3s(alpha,beta,kappa1,kappa2) + 3*G1s3s(alpha, beta, 1-kappa1, 1-kappa2))   # xxx and yyy
    elif beta == 0:
        if not check_duplicates([beta,kappa1,kappa2]):
            return sympy.simplify(6*G1s3s(alpha,beta,kappa1,kappa2))                                         # xyz
        elif kappa1 == 0:
            if kappa2 == 1:
                return sympy.simplify(6*G1s3s(alpha,beta,kappa1,kappa2) + 3*G1s3s(alpha,1,1,1) )             # xxy
            elif kappa2 ==2:
                return sympy.simplify(4*G1s3s(alpha,beta,kappa1,kappa2) + G1s3s(alpha,1,1,2)   )             # xxz
        elif kappa1 == 1:
            return sympy.simplify(6*G1s3s(alpha,beta,kappa1,kappa2) + 3*G1s3s(alpha,0,0,0) )                 # xyy
    elif beta ==1:
        return sympy.simplify(4*G1s3s(alpha,beta,kappa1,kappa2) + G1s3s(alpha,0,0,2) )                       # yyz
    
    
    

def symG2a2s(alpha, beta, kappa1):
    if beta == kappa1:
        return 0#sympy.simplify(3/2*G2a2s(alpha, beta, kappa1) - 3*G2a2s(alpha, 1-beta, 1-kappa1))
    else:
        return sympy.simplify(2*G2a2s(alpha, beta, kappa1))
    
def symG2a3a(alpha, beta, kappa1):
    if beta == kappa1:
        return 0#sympy.simplify(2*G2a3a(alpha, beta, kappa1) + G2a3a(alpha, 1-beta, 1-kappa1)) 
    else:
        return sympy.simplify(2*G2a3a(alpha, beta, kappa1))
    
def symG2a3s(alpha, beta, kappa1, kappa2):
    if all_equal([beta,kappa1,kappa2]):
        return sympy.simplify(4*G2a3s(alpha,beta,kappa1,kappa2) + 3*G2a3s(alpha, beta, 1-kappa1, 1-kappa2))   # xxx and yyy
    elif beta == 0:
        if not check_duplicates([beta,kappa1,kappa2]):
            return sympy.simplify(6*G2a3s(alpha,beta,kappa1,kappa2))                                         # xyz
        elif kappa1 == 0:
            if kappa2 == 1:
                return sympy.simplify(6*G2a3s(alpha,beta,kappa1,kappa2) + 3*G2a3s(alpha,1,1,1) )             # xxy
            elif kappa2 ==2:
                return sympy.simplify(4*G2a3s(alpha,beta,kappa1,kappa2) + G2a3s(alpha,1,1,2)   )             # xxz
        elif kappa1 == 1:
            return sympy.simplify(6*G2a3s(alpha,beta,kappa1,kappa2) + 3*G2a3s(alpha,0,0,0) )                 # xyy
    elif beta ==1:
        return sympy.simplify(4*G2a3s(alpha,beta,kappa1,kappa2) + G2a3s(alpha,0,0,2) )                       # yyz

In [27]:
%%time
for sldl in SLDL:
    if sldl =='G':
        for mode1 in modes:
            for mode2 in modes:
                if mode1 in ['1s', '2a']:
                    if mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()['sym'+sldl+mode1+mode2])(ki, kj, lj) for ki in range(3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

G1s2s done
G1s3a done
G2a2s done
G2a3a done
CPU times: user 7.14 s, sys: 8 ms, total: 7.15 s
Wall time: 7.23 s


### Matrix assembly

In [9]:
n_jobs = 1

In [10]:
modes = ['1s', '2a', '2s', '3t', '3a', '3s']
modesV = ['2s', '3t'] ##possible slip modes, for double-layer matrix elements
SLDL  = ['G']#, 'K']
dim   = {'1s':3, '2a':3, '2s':5, '3t':3, '3a':5, '3s':7}

In [11]:
%%time
for sldl in SLDL:
    if sldl =='G':
        for mode1 in modes:
            for mode2 in modes:
                if mode1 in ['1s', '2a']:
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj) for ki in range(3) 
                                                                            for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj, lj) for ki in range(3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj, lj, mj) for ki in range(3)
                                                                                            for kj in range(2)
                                                                                                for lj in range(kj,2)
                                                                                                    for mj in range(lj,3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')
                        
                        
                        
                elif mode1 == '3t':
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj) for ki in range(3) 
                                                                            for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj, lj) for ki in range(3)
                                                                                for kj in range(2)
                                                                                    for lj in range(kj,3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, kj, lj, mj) for ki in range(3)
                                                                                    for kj in range(2)
                                                                                        for lj in range(kj,2)
                                                                                            for mj in range(lj,3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')



                elif mode1 in ['2s', '3a']:
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, kj) for ki in range(2) 
                                                                                for li in range(ki,3)
                                                                                    for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, kj, lj) for ki in range(2)
                                                                                    for li in range(ki,3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, kj, lj, mj) for ki in range(2)
                                                                                        for li in range(ki,3)
                                                                                            for kj in range(2)
                                                                                                for lj in range(kj,2)
                                                                                                    for mj in range(lj,3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')



                elif mode1 == '3s':
                    if mode2 in ['1s', '2a', '3t']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj) for ki in range(2) 
                                                                                    for li in range(ki,2)
                                                                                        for mi in range(li,3)
                                                                                            for kj in range(3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 in ['2s', '3a']:
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj, lj) for ki in range(2)
                                                                                        for li in range(ki,2)
                                                                                            for mi in range(li,3)
                                                                                                for kj in range(2)
                                                                                                    for lj in range(kj,3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

                    elif mode2 =='3s':
                        globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                            delayed(globals()[sldl+mode1+mode2])(ki, li, mi, kj, lj, mj) for ki in range(2)
                                                                                            for li in range(ki,2)
                                                                                                for mi in range(li,3)
                                                                                                    for kj in range(2)
                                                                                                        for lj in range(kj,2)
                                                                                                            for mj in range(lj,3))
                        globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                        globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                        print(sldl+mode1+mode2+' done')

G1s1s done
G1s2a done
G1s2s done
G1s3t done
G1s3a done
G1s3s done
G2a1s done
G2a2a done
G2a2s done
G2a3t done
G2a3a done
G2a3s done
G2s1s done
G2s2a done
G2s2s done
G2s3t done
G2s3a done
G2s3s done
G3t1s done
G3t2a done
G3t2s done
G3t3t done
G3t3a done
G3t3s done
G3a1s done
G3a2a done
G3a2s done
G3a3t done
G3a3a done
G3a3s done
G3s1s done
G3s2a done
G3s2s done
G3s3t done
G3s3a done
G3s3s done
CPU times: user 6min 21s, sys: 145 ms, total: 6min 21s
Wall time: 6min 21s


In [12]:
## create double-layer matrices
for mode1 in modes:
    for mode2 in modesV:
        if mode2 == '2s':
            globals()['matK'+mode1+mode2] = 8*pi*eta*b/3 * globals()['matG'+mode1+mode2]
            print('K'+mode1+mode2+' done')
            
        elif mode2 == '3t':
            globals()['matK'+mode1+mode2] = 6*pi*eta*b/5 * globals()['matG'+mode1+mode2]
            print('K'+mode1+mode2+' done')

K1s2s done
K1s3t done
K2a2s done
K2a3t done
K2s2s done
K2s3t done
K3t2s done
K3t3t done
K3a2s done
K3a3t done
K3s2s done
K3s3t done


In [13]:
## Delta tensors as 'identity' matrices
def Delta1(alpha, beta):
    return KroneckerDelta[alpha,beta]

def Delta2(alpha, gamma1, beta, kappa1):
    return (1/2*KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa1] 
            + 1/2*KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,beta]
            -1/3*KroneckerDelta[alpha,gamma1]*KroneckerDelta[beta,kappa1])

def Delta3(alpha, gamma1, gamma2, beta, kappa1, kappa2):
    return 1/6*(KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa1]*KroneckerDelta[gamma2,kappa2] 
                + KroneckerDelta[alpha,beta]*KroneckerDelta[gamma1,kappa2]*KroneckerDelta[gamma2,kappa1] 
                + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,beta]*KroneckerDelta[gamma2,kappa2] 
                + KroneckerDelta[alpha,kappa1]*KroneckerDelta[gamma1,kappa2]*KroneckerDelta[gamma2,beta] 
                + KroneckerDelta[alpha,kappa2]*KroneckerDelta[gamma1,beta]*KroneckerDelta[gamma2,kappa1]
                + KroneckerDelta[alpha,kappa2]*KroneckerDelta[gamma1,kappa1]*KroneckerDelta[gamma2,beta]) - 1/15*(
        (KroneckerDelta[beta,kappa1]*KroneckerDelta[gamma2,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[gamma2,kappa1]
         + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[gamma2,beta])*KroneckerDelta[alpha,gamma1]
        +(KroneckerDelta[beta,kappa1]*KroneckerDelta[alpha,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[alpha,kappa1]
          + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[alpha,beta])*KroneckerDelta[gamma1,gamma2]
        +(KroneckerDelta[beta,kappa1]*KroneckerDelta[gamma1,kappa2] + KroneckerDelta[beta,kappa2]*KroneckerDelta[gamma1,kappa1]
          + KroneckerDelta[kappa1,kappa2]*KroneckerDelta[gamma1,beta])*KroneckerDelta[alpha,gamma2])

In [104]:
listDelta1 = Parallel(n_jobs=n_jobs)(delayed(Delta1)(ki,kj) for ki in range(3)
                                                                            for kj in range(3))
arrayDelta1 = np.asarray(listDelta1).reshape(3,3)
matDelta1   = sympy.Matrix(arrayDelta1)

##
listDelta2 = Parallel(n_jobs=n_jobs)(delayed(Delta2)(ki,li, kj,lj) for ki in range(2)
                                                                        for li in range(ki,3)
                                                                            for kj in range(2)
                                                                              for lj in range(kj,3))
arrayDelta2 = np.asarray(listDelta2).reshape(5,5)
matDelta2   = sympy.nsimplify(sympy.Matrix(arrayDelta2), rational=True)

##
listDelta3 = Parallel(n_jobs=n_jobs)(delayed(Delta3)(ki,li,mi, kj,lj,mj) for ki in range(2)
                                                                                for li in range(ki,2)
                                                                                    for mi in range(li,3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,2)
                                                                                                for mj in range(lj,3))
arrayDelta3 = np.asarray(listDelta3).reshape(7,7)
matDelta3   = sympy.nsimplify(sympy.Matrix(arrayDelta3), rational=True) ## rationals do not alter results in simulation...

In [106]:
## In python code will be force[i,0], so have to create force as np.zeros([3*Np,1]) object instead of np.zeros([3*Np])
force  = sympy.Matrix(sympy.MatrixSymbol('force',3,1)) ##create an actual Matrix from it, have to pass to 'Matrix' 
torque = sympy.Matrix(sympy.MatrixSymbol('torque',3,1))
VH     = sympy.Matrix(sympy.BlockDiagMatrix(matDelta2.inv(), matDelta1.inv(), matDelta2.inv(), matDelta3.inv()))*sympy.Matrix(sympy.MatrixSymbol('VH',20,1))
FH     = sympy.Matrix(sympy.BlockDiagMatrix(matDelta2.inv(), matDelta1.inv(), matDelta2.inv(), matDelta3.inv()))*sympy.Matrix(sympy.MatrixSymbol('FH',20,1))

In [112]:
VH

Matrix([
[    2*VH[0, 0] + VH[3, 0]],
[               2*VH[1, 0]],
[               2*VH[2, 0]],
[    VH[0, 0] + 2*VH[3, 0]],
[               2*VH[4, 0]],
[                 VH[5, 0]],
[                 VH[6, 0]],
[                 VH[7, 0]],
[   2*VH[8, 0] + VH[11, 0]],
[               2*VH[9, 0]],
[              2*VH[10, 0]],
[   VH[8, 0] + 2*VH[11, 0]],
[              2*VH[12, 0]],
[4*VH[13, 0] + 3*VH[16, 0]],
[6*VH[14, 0] + 3*VH[18, 0]],
[  4*VH[15, 0] + VH[19, 0]],
[3*VH[13, 0] + 6*VH[16, 0]],
[              6*VH[17, 0]],
[3*VH[14, 0] + 4*VH[18, 0]],
[  VH[15, 0] + 4*VH[19, 0]]])

Create block matrices, then multiply with vectors and then print as functions

In [107]:
## need numpy diagonals and convert them to Matrix object for GoHH and KoHH

In [108]:
matG1sH = sympy.Matrix(sympy.BlockMatrix([matG1s2s, matG1s3t, matG1s3a, matG1s3s]))
matG2aH = sympy.Matrix(sympy.BlockMatrix([matG2a2s, matG2a3t, matG2a3a, matG2a3s]))

matK1sH = sympy.Matrix(sympy.BlockMatrix([matK1s2s, matK1s3t, sympy.Matrix(sympy.ZeroMatrix(3,12))]))
matK2aH = sympy.Matrix(sympy.BlockMatrix([matK2a2s, matK2a3t, sympy.Matrix(sympy.ZeroMatrix(3,12))]))

matGH1s = sympy.Matrix(sympy.BlockMatrix([[matG2s1s],
                              [matG3t1s],
                              [matG3a1s],
                              [matG3s1s]]))
matGH2a = sympy.Matrix(sympy.BlockMatrix([[matG2s2a],
                              [matG3t2a],
                              [matG3a2a],
                              [matG3s2a]]))

matGHH = sympy.Matrix(sympy.BlockMatrix([[matG2s2s, matG2s3t, matG2s3a, matG2s3s],
                             [matG3t2s, matG3t3t, matG3t3a, matG3t3s],
                             [matG3a2s, matG3a3t, matG3a3a, matG3a3s],
                             [matG3s2s, matG3s3t, matG3s3a, matG3s3s]]))

matKHH_nonzero = sympy.Matrix(sympy.BlockMatrix([[matK2s2s, matK2s3t],
                                     [matK3t2s, matK3t3t],
                                     [matK3a2s, matK3a3t],
                                     [matK3s2s, matK3s3t]]))
matKHH = sympy.Matrix(sympy.BlockMatrix([matKHH_nonzero, sympy.Matrix(sympy.ZeroMatrix(20,12))]))

# self-interaction

g2s = 3/(20*pi*eta*b)
g3t = 1/(2*pi*eta*b)
g3a = 3/(2*pi*eta*b)
g3s = 6/(7*pi*eta*b)

matGoHH = sympy.Matrix(sympy.BlockDiagMatrix(g2s*matDelta2,
                                             g3t*matDelta1,
                                             g3a*matDelta2,
                                             g3s*matDelta3))


# arrayGoHH = np.diag(np.block([np.full(5, g2s),
#                               np.full(3, g3t),
#                               np.full(5, g3a),
#                               np.full(7, g3s)]))

# matGoHH = sympy.Matrix(arrayGoHH)

halfMinusk2s = 3/5 #0.6
halfMinusk3t = 2/5 #0.4
halfMinusk3a = 4/5 #0.8
halfMinusk3s = 19/35

matKoHH = sympy.Matrix(sympy.BlockDiagMatrix(halfMinusk2s*matDelta2,
                                             halfMinusk3t*matDelta1,
                                             halfMinusk3a*matDelta2,
                                             halfMinusk3s*matDelta3))

# arrayKoHH = np.diag(np.block([np.full(5, halfMinusk2s),
#                               np.full(3, halfMinusk3t),
#                               np.full(5, halfMinusk3a),
#                               np.full(7, halfMinusk3s)]))

# matKoHH = sympy.Matrix(arrayKoHH)

Create vectors instead of constructing the full matrix

In [109]:
vecGH1sF = matGH1s*force
vecGH2aT = matGH2a*torque

vecKoHHVH = matKoHH*VH
vecKHHVH = matKHH*VH

vecGoHHFH = matGoHH*FH
vecGHHFH = matGHH*FH

vecG1s1sF = matG1s1s*force
vecG2a1sF = matG2a1s*force

vecG1s2aT = matG1s2a*torque
vecG2a2aT = matG2a2a*torque

vecG1sHFH = matG1sH*FH
vecG2aHFH = matG2aH*FH

vecK1sHVH = matK1sH*VH
vecK2aHVH = matK2aH*VH

_____________

### Write symbolic results to file as reusable python functions

In [110]:
ME_file = "symME.py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s1sF),(3,)).tolist()), file=text_file)
    print("def G2a1sF(xij,yij,zij, b,eta, force):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a1sF),(3,)).tolist()), file=text_file)
    
    print("def G1s2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1s2aT),(3,)).tolist()), file=text_file)
    print("def G2a2aT(xij,yij,zij, b,eta, torque):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2a2aT),(3,)).tolist()), file=text_file)
    
    print("def GH1sF(gh1sf, dimH, i, xij,yij,zij, b,eta, force):", file=text_file)
    print("    gh1sf[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecGH1sF),(20,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GH2aT(gh2at, dimH, i, xij,yij,zij, b,eta, torque):", file=text_file)
    print("    gh2at[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecGH2aT),(20,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KoHHVH(khhvh, dimH, i, b,eta, VH):", file=text_file)
    print("    khhvh[dimH*i:dimH*(i+1)] -= {}".format(np.reshape(np.asarray(vecKoHHVH),(20,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def KHHVH(khhvh, dimH, i, xij,yij,zij, b,eta, VH):", file=text_file)
    print("    khhvh[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecKHHVH),(20,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GoHHFH(ghhfh, dimH, i, b,eta, FH):", file=text_file)
    print("    ghhfh[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecGoHHFH),(20,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def GHHFH(ghhfh, dimH, i, xij,yij,zij, b,eta, FH):", file=text_file)
    print("    ghhfh[dimH*i:dimH*(i+1)] += {}".format(np.reshape(np.asarray(vecGHHFH),(20,)).tolist()), file=text_file)
    print("    return\n", file=text_file)
    
    print("def G1sHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG1sHFH),(3,)).tolist()), file=text_file)
    print("def G2aHFH(xij,yij,zij, b,eta, FH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecG2aHFH),(3,)).tolist()), file=text_file)
    
    print("def K1sHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK1sHVH),(3,)).tolist()), file=text_file)
    print("def K2aHVH(xij,yij,zij, b,eta, VH):\n    return numpy.array({})\n".format(np.reshape(np.asarray(vecK2aHVH),(3,)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

filedata = re.sub("force\[(\d), 0\]", r"force[\1]", filedata)
filedata = re.sub("torque\[(\d), 0\]", r"torque[\1]", filedata)
filedata = re.sub("VH\[(\d||\d\d), 0\]", r"VH[\1]", filedata)
filedata = re.sub("FH\[(\d||\d\d), 0\]", r"FH[\1]", filedata)

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________

In [23]:
sympy.nsimplify(matDelta2*VH[:5,0])

Matrix([
[ 2*VH[0, 0]/3 - VH[3, 0]/3],
[                VH[1, 0]/2],
[                VH[2, 0]/2],
[-VH[0, 0]/3 + 2*VH[3, 0]/3],
[                VH[4, 0]/2]])

In [44]:
sympy.simplify(sympy.nsimplify((matK1s2s*matDelta2.inv()*VH[:5,0]).subs({xij:3, yij:0, zij:0})))

Matrix([
[b**2*(45 - 8*b**2)*VH[0, 0]/405],
[          16*b**4*VH[1, 0]/1215],
[          16*b**4*VH[2, 0]/1215]])

In [53]:
matDelta2.inv()*FH[:5,0]

Matrix([
[2.0*FH[0, 0] + 0.999999999999997*FH[3, 0]],
[                             2.0*FH[1, 0]],
[                             2.0*FH[2, 0]],
[0.999999999999997*FH[0, 0] + 2.0*FH[3, 0]],
[                             2.0*FH[4, 0]]])

_____________

### Save **matrix** expressions for **direct** solver 

This is to eliminate one variable in the uncertainty whether the results are correct/incorrect. Integrator + Krylov seems very touchy

In [17]:
ME_file = "matrixME.py"

with open(ME_file, "w") as text_file:
    print("import numpy", file=text_file)
    print("PI = 3.14159265359\n", file=text_file)
    
    print("def G1s1s(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s1s),(3,3)).tolist()), file=text_file)
    print("def G2a1s(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a1s),(3,3)).tolist()), file=text_file)
    
    print("def G1s2a(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1s2a),(3,3)).tolist()), file=text_file)
    print("def G2a2a(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2a2a),(3,3)).tolist()), file=text_file)
    
    print("def GH1s(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH1s),(25,3)).tolist()), file=text_file)
    print("def GH2a(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGH2a),(25,3)).tolist()), file=text_file)
    
    print("def KoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKoHH),(25,25)).tolist()), file=text_file)
    print("def KHH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matKHH),(25,25)).tolist()), file=text_file)
    
    print("def GoHH(b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGoHH),(25,25)).tolist()), file=text_file)
    print("def GHH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matGHH),(25,25)).tolist()), file=text_file)
    
    print("def G1sH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG1sH),(3,25)).tolist()), file=text_file)
    print("def G2aH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matG2aH),(3,25)).tolist()), file=text_file)
    
    print("def K1sH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK1sH),(3,25)).tolist()), file=text_file)
    print("def K2aH(xij,yij,zij, b,eta):\n    return numpy.array({})\n".format(np.reshape(np.asarray(matK2aH),(3,25)).tolist()), file=text_file)
    
    
    
## replace things for easier usage with python     
with open(ME_file, 'r') as file:
    filedata = file.read()
    
filedata = filedata.replace('pi', 'PI')
filedata = filedata.replace('sqrt', 'numpy.sqrt')

with open(ME_file, 'w') as file:
    file.write(filedata)

______________________________________
_______________________

In [22]:
matK3a2s[0,1].subs({xij:0,yij:0, zij:2.3})/2

0.0357345778495646*b**4